In [3]:
'''
writing a classifier to classify topics
Using the reuters dataset from keras dataset for that
'''

'\nwriting a classifier to classify topics\nUsing the reuters dataset from keras dataset for that\n'

In [1]:
import keras
from keras.datasets import reuters

In [ ]:
# Not putting a cap on the most frequently used word from the dataset
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=None, )